In [17]:
import re
import time
import json
import requests
import datetime
import pandas as pd

from pprint import pprint
from bs4 import BeautifulSoup

In [18]:
headers = {
'Host' : 'auto.ru',
'User-Agent' : 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0',
'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
'Accept-Language' : 'en-US,en;q=0.5',
'Accept-Encoding' : 'gzip, deflate, br',
'Connection' : 'keep-alive',
'Cookie' : 'autoru_gdpr=1; suid=bc8be712d638a08131a6e9e0d202d740.c5b54afb905b8c06122ab3a791bcd52b; yandexuid=2760879651631376416; my=YysBAgA%3D; counter_ga_all7=2; credit_filter_promo_popup_closed=true; bltsr=1; autoru-visits-count=5; autoru_sid=a%3Ag619130a126cnq19ulaaft362m5p2vb2.ce95d73e09b71591f2b431378fbd3c26%7C1637509921806.604800.1BSyVNdQ_Jgiakk7ftwSsg.rxF87OjlP3hNehiaJwY7hz1CT9Y7Izf61mSqV5Ly0C8; autoruuid=g619130a126cnq19ulaaft362m5p2vb2.ce95d73e09b71591f2b431378fbd3c26; crookie=jpj0jVvWTMkqF2bLNEjq1Sm8BK5liRI8I2IDRbmuxJB1Z9qhACbqZpnyGeU1riNvks6acZdg/tULhikIF+VQKxi5Ddo=; deal_million_popup_page_seen=-1; credit_modal_autoshow_closed=true; _yasc=BXlfPgARlvaqwGCFNYlOD81UvI/rudyiPXKWSUFuIbFRxQ==; _csrf_token=cd33b362bade8bf97053fcccb6e8a730d67cea3890bc13dd; from_lifetime=1637870096528; from=direct; safe_deal_promo=0; yuidlt=1; cmtchd=MTYzNzg2MjAxMjI2OQ==; X-Vertis-DC=sas; los=1',
'Upgrade-Insecure-Requests' : '1',
'Sec-Fetch-Dest' : 'document',
'Sec-Fetch-Mode' : 'navigate',
'Sec-Fetch-Site' : 'none',
'Sec-Fetch-User' : '?1'
}
page_suffix = '?page='

In [19]:
perks = ['Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'bodyType', 'brand', 'color', 'complectation_dict', 'description', 'engineDisplacement', 
      'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 
      'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 
      'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'car_url']

In [20]:
def get_ad_info(url, headers, characteristics):
    ad_info = dict()   
    r = requests.get(url, headers=headers)
    html = BeautifulSoup(r.content, 'html.parser')
    for li in html.find_all('li', 'CardInfoRow'):
        var1, var2 = tuple(li.children)
        key = var1.string
        value = var2.string
        if key in perks[:7]:
            ad_info[key] = value
    
    try:
        j1 = json.loads(html.script.string)
    except:
        return
    
    try:
        j2 = json.loads(html.find('script', id="initial-state").string)
    except:
        return
        
    for k in set(perks).intersection(set(j1)):
        ad_info[k] = j1[k]

    for l in set(perks).intersection(j1['vehicleEngine']):
        try:
            ad_info[l] = j1['vehicleEngine'][l]
        except:
            pass

    for m in set(perks).intersection(j2['card']['state']):
        try:
            ad_info[m] = j2['card']['state'][m]
        except:
            pass

    try: 
        ad_info['equipment_dict'] = j2['card']['vehicle_info']['equipment'] 
    except: 
        pass
    
    try: 
        ad_info['complectation_dict'] = j2['card']['vehicle_info']['complectation'] 
    except: 
        pass
    
    try: 
        ad_info['model_info'] = j2['card']['vehicle_info']['model_info'] 
    except: 
        pass
    
    try: 
        ad_info['model_name'] = j2['card']['vehicle_info']['model_info']['name'] 
    except: 
        pass
    
    try: 
        ad_info['super_gen'] = j2['card']['vehicle_info']['super_gen'] 
    except: 
        pass
    
    try: 
        ad_info['vendor'] = j2['card']['vehicle_info']['vendor'] 
    except: 
        pass
    
    try: 
        ad_info['sell_id'] = j2['card']['saleId'] 
    except: 
        pass
    

    try: 
        ad_info['price'] = j1['offers']['price'] 
    except: 
        pass
    
    try: 
        ad_info['priceCurrency'] = j1['offers']['priceCurrency'] 
    except: 
        pass
    
    try: 
        ad_info['car_url'] = j1['offers']['url'] 
    except: 
        pass
    

    ad_info['parsing_unixtime'] = time.time()

    return ad_info

In [ ]:
time_start = datetime.datetime.now()
makes = ['', 'renault', 'toyota', 'mercedes', 'audi', 'hyundai', 'ford']
makes = ['vaz', 'chevrolet', 'mitsubishi', 'volkswagen']
for make in makes:
    df = pd.DataFrame(columns=perks)
    print(f'{make=}')
    url = 'https://auto.ru/cars/' + make + '/all/'
    rng = range(1, 100)
    for page_num in rng:
        if page_num <=1:
            page_url = url
        else:
            page_url = url + page_suffix + str(page_num)
        res = requests.get(page_url, headers=headers)
        code = BeautifulSoup(res.content, 'html.parser')
        ad_links = code.find_all("a", class_="ListingItemTitle__link")
        for l in ad_links:
            link = l['href']
            print(f'{page_num = }  __  {link = }')
            car_info = get_ad_info(link, headers, perks)
            if car_info is None:
                pass
            else:
                df = df.append(car_info, ignore_index=True,)
            time.sleep(0.5)
    time_passed = datetime.datetime.now() - time_start
    print(f'{time_passed=}')
    df.to_csv('project6' + make + '.csv')

# Работа с данными

In [244]:
df = pd.read_csv('project6.csv')
df = df.append(pd.read_csv('project6audi.csv'))
df = df.append(pd.read_csv('project6_ford_1.csv'))
df = df.append(pd.read_csv('project6_ford_3.csv'))
df = df.append(pd.read_csv('project6_ford_4.csv'))
df = df.append(pd.read_csv('project6hyundai.csv'))
df = df.append(pd.read_csv('project6mercedes.csv'))
df = df.append(pd.read_csv('project6renault.csv'))
df = df.append(pd.read_csv('project6toyota.csv'))
df = df.append(pd.read_csv('project6vaz.csv'))
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28673 entries, 0 to 28672
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 28673 non-null  int64  
 1   Unnamed: 0            28673 non-null  int64  
 2   Владельцы             21776 non-null  object 
 3   Владение              3273 non-null   object 
 4   ПТС                   21776 non-null  object 
 5   Привод                21776 non-null  object 
 6   Руль                  21776 non-null  object 
 7   Состояние             21776 non-null  object 
 8   Таможня               21776 non-null  object 
 9   bodyType              28673 non-null  object 
 10  brand                 28673 non-null  object 
 11  color                 28673 non-null  object 
 12  complectation_dict    28670 non-null  object 
 13  description           28673 non-null  object 
 14  engineDisplacement    28673 non-null  object 
 15  enginePower        

In [245]:
df.columns

Index(['index', 'Unnamed: 0', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня', 'bodyType', 'brand', 'color',
       'complectation_dict', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage',
       'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors',
       'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id',
       'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor',
       'car_url', 'price'],
      dtype='object')

In [246]:
df['Владельцы'].value_counts()

1 владелец     8520
3 или более    8182
2 владельца    5074
Name: Владельцы, dtype: int64

In [247]:
df['Владельцы'] = df['Владельцы'].apply(lambda x: 1 if str(x).startswith('1') else 2 if str(x).startswith('2') else 3 if str(x).startswith('3') else x)

In [248]:
def time_owned(text):
    years_re = re.compile('\d+(?= год| лет)')
    month_re = re.compile('\d+(?= месяц)')
    years = years_re.search(text)
    months = month_re.search(text)
    if years is None and months is None:
        return None
    if years is None:
        years = 0
    else:
        years = int(years[0])
    if months is None:
        months = 0
    else:
        months = int(months[0])
    return years * 12 + months

In [249]:
df['Владение'].dropna().apply(time_owned).index

Int64Index([    1,     4,     5,    12,    13,    17,    22,    25,    28,
               30,
            ...
            28260, 28418, 28504, 28530, 28556, 28580, 28581, 28587, 28608,
            28637],
           dtype='int64', length=3273)

In [250]:
df['ПТС'].apply(lambda x: 1 if x == 'Оригинал' else 0)

0        0
1        1
2        0
3        0
4        0
        ..
28668    1
28669    1
28670    1
28671    1
28672    0
Name: ПТС, Length: 28673, dtype: int64

In [251]:
df['Привод'].value_counts()

передний    12719
полный       7751
задний       1306
Name: Привод, dtype: int64

In [252]:
df['Руль'].apply(lambda x: 1 if x == 'Левый' else 0)

0        0
1        1
2        1
3        0
4        0
        ..
28668    1
28669    1
28670    1
28671    1
28672    0
Name: Руль, Length: 28673, dtype: int64

In [253]:
df['Состояние'].apply(lambda x: 1 if x == 'Не требует ремонта' else 0)

0        0
1        1
2        1
3        0
4        1
        ..
28668    1
28669    1
28670    1
28671    1
28672    0
Name: Состояние, Length: 28673, dtype: int64

In [254]:
df['Таможня'].apply(lambda x: 1 if x == 'Растаможен' else 0)

0        0
1        1
2        1
3        0
4        1
        ..
28668    1
28669    1
28670    1
28671    1
28672    0
Name: Таможня, Length: 28673, dtype: int64

In [255]:
df['bodyType'].value_counts()

внедорожник 5 дв.          11019
седан                       9896
хэтчбек 5 дв.               3184
универсал 5 дв.             1680
лифтбек                      687
минивэн                      563
купе                         430
хэтчбек 3 дв.                307
компактвэн                   300
внедорожник 3 дв.            240
пикап двойная кабина         153
фургон                        85
купе-хардтоп                  52
кабриолет                     34
родстер                       17
пикап полуторная кабина        7
седан-хардтоп                  4
пикап одинарная кабина         4
микровэн                       4
хэтчбек 4 дв.                  3
внедорожник открытый           2
лимузин                        2
Name: bodyType, dtype: int64

In [256]:
pd.concat([df, pd.get_dummies(df['bodyType'])], axis=1)

,index,Unnamed: 0,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,...,пикап одинарная кабина,пикап полуторная кабина,родстер,седан,седан-хардтоп,универсал 5 дв.,фургон,хэтчбек 3 дв.,хэтчбек 4 дв.,хэтчбек 5 дв.
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1.0,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,...,0,0,0,0,0,0,0,0,0,0
2,2,2,3.0,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,1,0,0,0,0,0,0
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,...,0,0,0,0,0,0,0,0,0,0
4,4,4,3.0,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,3642,3642,2.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,0,0,0,0,1
28669,3643,3643,3.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,0,0,0,0,1
28670,3644,3644,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,1,0,0,0,0,0,0
28671,3645,3645,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,1,0,0,0,0,0,0


In [257]:
df['brand'].value_counts()

HYUNDAI        4152
VAZ            3941
RENAULT        3887
TOYOTA         3865
MERCEDES       3815
               ... 
IG                1
GAC               1
LAMBORGHINI       1
CHANGAN           1
ACURA             1
Name: brand, Length: 64, dtype: int64

In [258]:
pd.concat([df, pd.get_dummies(df['brand'])], axis=1)

,index,Unnamed: 0,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,...,SUBARU,SUZUKI,TESLA,TOYOTA,UAZ,VAZ,VOLKSWAGEN,VOLVO,VORTEX,ZAZ
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1.0,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,...,0,0,0,1,0,0,0,0,0,0
2,2,2,3.0,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,0,0,0,0,0,0,0
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,...,0,0,0,0,0,0,0,0,0,0
4,4,4,3.0,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,3642,3642,2.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,1,0,0,0,0
28669,3643,3643,3.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,1,0,0,0,0
28670,3644,3644,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,0,0,1,0,0,0,0
28671,3645,3645,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,0,0,1,0,0,0,0


In [259]:
df['color'].value_counts()

чёрный         6875
белый          6562
серый          4292
серебристый    2847
синий          2569
красный        1524
коричневый     1430
зелёный         732
бежевый         613
оранжевый       428
голубой         331
пурпурный       144
фиолетовый      123
золотистый      102
жёлтый           89
розовый          12
Name: color, dtype: int64

In [260]:
pd.concat([df, pd.get_dummies(df['color'])], axis=1)

,index,Unnamed: 0,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,...,коричневый,красный,оранжевый,пурпурный,розовый,серебристый,серый,синий,фиолетовый,чёрный
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1.0,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,...,0,0,0,1,0,0,0,0,0,0
2,2,2,3.0,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,0,0,0,1,0,0,0
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,...,0,0,0,0,0,1,0,0,0,0
4,4,4,3.0,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,3642,3642,2.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,0,0,0,0,1
28669,3643,3643,3.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,0,0,0,0,0,1,0,0,0,0
28670,3644,3644,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,0,0,0,0,0,0,0,0,1
28671,3645,3645,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,0,1,0,0,0,0,0,0,0,0


In [261]:
df['engineDisplacement'].value_counts()

1.6 LTR    9786
2.0 LTR    6210
3.0 LTR    1757
2.5 LTR    1420
1.8 LTR    1273
1.4 LTR    1178
1.5 LTR    1128
3.5 LTR     784
1.3 LTR     745
4.0 LTR     532
2.4 LTR     362
2.8 LTR     346
2.9 LTR     341
1.7 LTR     316
2.2 LTR     298
2.1 LTR     275
2.3 LTR     272
4.5 LTR     220
2.7 LTR     216
5.5 LTR     178
4.7 LTR     163
 LTR         95
1.2 LTR      85
4.2 LTR      81
4.6 LTR      63
3.2 LTR      52
1.0 LTR      43
5.0 LTR      39
3.1 LTR      39
3.6 LTR      32
5.4 LTR      30
2.6 LTR      28
0.8 LTR      25
3.7 LTR      25
1.9 LTR      24
5.7 LTR      24
3.4 LTR      20
6.2 LTR      20
3.8 LTR      18
6.0 LTR      16
3.3 LTR      14
5.2 LTR      12
4.1 LTR      12
1.1 LTR      11
4.4 LTR       9
5.9 LTR       9
5.6 LTR       7
5.3 LTR       7
5.8 LTR       6
0.7 LTR       5
4.9 LTR       5
4.3 LTR       5
6.8 LTR       5
6.3 LTR       4
6.1 LTR       3
Name: engineDisplacement, dtype: int64

In [262]:
df['engineDisplacement'].apply(lambda x: float(str(x).replace('LTR', '')) if str(x).replace('LTR', '').strip() != '' else None).value_counts()

1.6    9786
2.0    6210
3.0    1757
2.5    1420
1.8    1273
1.4    1178
1.5    1128
3.5     784
1.3     745
4.0     532
2.4     362
2.8     346
2.9     341
1.7     316
2.2     298
2.1     275
2.3     272
4.5     220
2.7     216
5.5     178
4.7     163
1.2      85
4.2      81
4.6      63
3.2      52
1.0      43
5.0      39
3.1      39
3.6      32
5.4      30
2.6      28
3.7      25
0.8      25
5.7      24
1.9      24
6.2      20
3.4      20
3.8      18
6.0      16
3.3      14
4.1      12
5.2      12
1.1      11
4.4       9
5.9       9
5.3       7
5.6       7
5.8       6
6.8       5
4.3       5
4.9       5
0.7       5
6.3       4
6.1       3
Name: engineDisplacement, dtype: int64

In [263]:
df['enginePower'].apply(lambda x: x.split()[0]).value_counts()

150    2359
249    1982
123    1912
106    1266
114     914
       ... 
318       1
646       1
608       1
227       1
40        1
Name: enginePower, Length: 297, dtype: int64

In [264]:
df['fuelType'].value_counts()

бензин     24563
дизель      3906
гибрид        98
электро       95
газ           11
Name: fuelType, dtype: int64

In [265]:
pd.concat([df, pd.get_dummies(df['fuelType'])], axis=1)

,index,Unnamed: 0,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,...,vehicleConfiguration,vehicleTransmission,vendor,car_url,price,бензин,газ,гибрид,дизель,электро
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,...,COUPE AUTOMATIC 2.0,автоматическая,EUROPEAN,https://auto.ru/cars/new/group/mercedes/e_klas...,5360064.0,1,0,0,0,0
1,1,1,1.0,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,...,ALLROAD_5_DOORS AUTOMATIC 2.5,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/rav_4/11...,2295000.0,1,0,0,0,0
2,2,2,3.0,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,...,SEDAN MECHANICAL 1.6,механическая,EUROPEAN,https://auto.ru/cars/used/sale/renault/megane/...,320000.0,1,0,0,0,0
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,...,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,KOREAN,https://auto.ru/cars/new/group/hyundai/creta/2...,1520000.0,1,0,0,0,0
4,4,4,3.0,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,HATCHBACK_5_DOORS AUTOMATIC 1.0,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/vitz/110...,175000.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,3642,3642,2.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,HATCHBACK_5_DOORS ROBOT 1.8,роботизированная,RUSSIAN,https://auto.ru/cars/used/sale/vaz/xray/110608...,554990.0,1,0,0,0,0
28669,3643,3643,3.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,HATCHBACK_5_DOORS MECHANICAL 1.6,механическая,RUSSIAN,https://auto.ru/cars/used/sale/vaz/2170/110606...,309000.0,1,0,0,0,0
28670,3644,3644,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,SEDAN ROBOT 1.6,роботизированная,RUSSIAN,https://auto.ru/cars/used/sale/vaz/vesta/11060...,624000.0,1,0,0,0,0
28671,3645,3645,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,SEDAN MECHANICAL 1.6,механическая,RUSSIAN,https://auto.ru/cars/used/sale/vaz/granta/1106...,698000.0,1,0,0,0,0


In [266]:
df['mileage'].apply(lambda x: None if x == 0 else x).value_counts()

200000.0    191
150000.0    167
120000.0    130
160000.0    130
180000.0    130
           ... 
159087.0      1
231301.0      1
107154.0      1
181754.0      1
102784.0      1
Name: mileage, Length: 10478, dtype: int64

In [23]:
df['modelDate'].value_counts()

0    2011
1    2012
2    2009
3    2020
4    2014
Name: modelDate, dtype: object

In [268]:
df['model_info'].head()

0    {'code': 'E_KLASSE', 'name': 'E-Класс', 'ru_na...
1    {'code': 'RAV_4', 'name': 'RAV4', 'ru_name': '...
2    {'code': 'MEGANE', 'name': 'Megane', 'ru_name'...
3    {'code': 'CRETA', 'name': 'Creta', 'ru_name': ...
4    {'code': 'VITZ', 'name': 'Vitz', 'ru_name': 'В...
Name: model_info, dtype: object

In [269]:
df['model_name'].value_counts()

Focus       1736
Duster      1345
Solaris     1338
Creta       1137
Camry       1001
            ... 
Skyline        1
Teramont       1
QX60           1
Bentayga       1
Quoris         1
Name: model_name, Length: 619, dtype: int64

In [270]:
df['numberOfDoors'].value_counts()

5    17361
4    10203
3      563
2      546
Name: numberOfDoors, dtype: int64

In [271]:
df['productionDate'].head()

0    2020
1    2018
2    2006
3    2021
4    2000
Name: productionDate, dtype: int64

In [272]:
df['vehicleTransmission'].value_counts()
pd.concat([df, pd.get_dummies(df['vehicleTransmission'])], axis=1)

,index,Unnamed: 0,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,...,super_gen,vehicleConfiguration,vehicleTransmission,vendor,car_url,price,автоматическая,вариатор,механическая,роботизированная
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,...,"{'id': '22284156', 'name': 'V (W213, S213, C23...",COUPE AUTOMATIC 2.0,автоматическая,EUROPEAN,https://auto.ru/cars/new/group/mercedes/e_klas...,5360064.0,1,0,0,0
1,1,1,1.0,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,...,"{'id': '20692838', 'name': 'IV (CA40) Рестайли...",ALLROAD_5_DOORS AUTOMATIC 2.5,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/rav_4/11...,2295000.0,1,0,0,0
2,2,2,3.0,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,...,"{'id': '6321881', 'name': 'II', 'ru_name': '2'...",SEDAN MECHANICAL 1.6,механическая,EUROPEAN,https://auto.ru/cars/used/sale/renault/megane/...,320000.0,0,0,1,0
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,...,"{'id': '22913367', 'name': 'II', 'ru_name': '2...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,KOREAN,https://auto.ru/cars/new/group/hyundai/creta/2...,1520000.0,1,0,0,0
4,4,4,3.0,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,"{'id': '8306666', 'name': 'I (P10)', 'ru_name'...",HATCHBACK_5_DOORS AUTOMATIC 1.0,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/vitz/110...,175000.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,3642,3642,2.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,"{'id': '20497289', 'name': 'I', 'ru_name': '1'...",HATCHBACK_5_DOORS ROBOT 1.8,роботизированная,RUSSIAN,https://auto.ru/cars/used/sale/vaz/xray/110608...,554990.0,0,0,0,1
28669,3643,3643,3.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,"{'id': '2307319', 'name': 'I', 'ru_name': '1',...",HATCHBACK_5_DOORS MECHANICAL 1.6,механическая,RUSSIAN,https://auto.ru/cars/used/sale/vaz/2170/110606...,309000.0,0,0,1,0
28670,3644,3644,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,"{'id': '20417749', 'name': 'I', 'year_from': 2...",SEDAN ROBOT 1.6,роботизированная,RUSSIAN,https://auto.ru/cars/used/sale/vaz/vesta/11060...,624000.0,0,0,0,1
28671,3645,3645,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,"{'id': '21377296', 'name': 'I Рестайлинг', 'ru...",SEDAN MECHANICAL 1.6,механическая,RUSSIAN,https://auto.ru/cars/used/sale/vaz/granta/1106...,698000.0,0,0,1,0


In [273]:
df['vendor'].value_counts()
pd.concat([df, pd.get_dummies(df['vendor'])], axis=1)

,index,Unnamed: 0,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,...,vehicleTransmission,vendor,car_url,price,AMERICAN,CHINESE,EUROPEAN,JAPANESE,KOREAN,RUSSIAN
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,...,автоматическая,EUROPEAN,https://auto.ru/cars/new/group/mercedes/e_klas...,5360064.0,0,0,1,0,0,0
1,1,1,1.0,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,...,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/rav_4/11...,2295000.0,0,0,0,1,0,0
2,2,2,3.0,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,...,механическая,EUROPEAN,https://auto.ru/cars/used/sale/renault/megane/...,320000.0,0,0,1,0,0,0
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,...,автоматическая,KOREAN,https://auto.ru/cars/new/group/hyundai/creta/2...,1520000.0,0,0,0,0,1,0
4,4,4,3.0,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/vitz/110...,175000.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28668,3642,3642,2.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,роботизированная,RUSSIAN,https://auto.ru/cars/used/sale/vaz/xray/110608...,554990.0,0,0,0,0,0,1
28669,3643,3643,3.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,...,механическая,RUSSIAN,https://auto.ru/cars/used/sale/vaz/2170/110606...,309000.0,0,0,0,0,0,1
28670,3644,3644,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,роботизированная,RUSSIAN,https://auto.ru/cars/used/sale/vaz/vesta/11060...,624000.0,0,0,0,0,0,1
28671,3645,3645,1.0,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,седан,...,механическая,RUSSIAN,https://auto.ru/cars/used/sale/vaz/granta/1106...,698000.0,0,0,0,0,0,1


In [300]:
# смотрим пример словаря для complectation_dict
df['complectation_dict'][0]
# содержаение первой записи
d = {'id': '22482399', 'name': 'E 200 4MATIC Sport', 'available_options': ['cruise-control', 'e-adjustment-wheel', 'multi-wheel', 'auto-park', 'airbag-passenger', 'navigation', 'decorative-interior-lighting', 'isofix-front', 'bas', 'lock', 'electro-mirrors', 'mirrors-heat', 'volume-sensor', 'leather', 'glonass', 'start-stop-function', 'automatic-lighting-control', 'collision-prevention-assist', 'computer', 'high-beam-assist', 'seat-transformation', 'android-auto', 'airbag-side', 'electronic-gage-panel', 'electro-trunk', 'abs', 'wheel-leather', 'climate-control-2', 'auto-mirrors', 'asr', 'esp', 'usb', 'audiosystem-tv', 'front-centre-armrest', 'apple-carplay', 'front-seat-support', 'rear-camera', 'servo', 'drl', 'electro-window-back', 'park-assist-f', 'rear-seats-heat', '19-inch-wheels', 'driver-seat-updown', 'led-lights', 'park-assist-r', 'airbag-driver', 'isofix', 'aux', 'drive-mode-sys', 'electro-window-front', 'airbag-curtain', 'passenger-seat-electric', 'alarm', 'paint-metallic', 'leather-gear-stick', 'blind-spot', 'start-button', 'voice-recognition', 'audiosystem-cd', 'front-seats-heat', 'bluetooth', 'wheel-configuration2', 'wheel-configuration1', 'immo', '12v-socket', 'third-rear-headrest'], 'vendor_colors': [{'body_color_id': 22805390, 'mark_color_id': 22191469, 'name_ru': 'Designo Красная патагония', 'hex_codes': ['ED173D'], 'color_type': 'METALLIC', 'stock_color': {'hex_code': 'EE1D19', 'name_ru': 'красный'}, 'photos': [{'name': '34-front', 'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/orig', 'wizardv3mr': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/wizardv3mr', 'wizardv3': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/wizardv3', 'cattouch': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/cattouch', 'small': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/small', '320x240': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/320x240', '1200x900': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/1200x900', '1200x900n': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/1200x900n', 'thumb_m': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/thumb_m', 'full': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/full', '832x624': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/832x624', '456x342': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/456x342', '120x90': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/120x90', '92x69': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/92x69', 'islandoff': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/islandoff', 'thumb_s': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/thumb_s', 'thumb_s_2x': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/thumb_s_2x', 'auto_main': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/auto_main', 'image': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/image'}}, {'name': '34-back', 'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/orig', 'wizardv3mr': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/wizardv3mr', 'wizardv3': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/wizardv3', 'cattouch': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/cattouch', 'small': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/small', '320x240': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/320x240', '1200x900': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/1200x900', '1200x900n': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/1200x900n', 'thumb_m': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/thumb_m', 'full': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/full', '832x624': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/832x624', '456x342': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/456x342', '120x90': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/120x90', '92x69': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/92x69', 'islandoff': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/islandoff', 'thumb_s': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/thumb_s', 'thumb_s_2x': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/thumb_s_2x', 'auto_main': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/auto_main', 'image': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aed50786cd1db5fce0963896/image'}}, {'name': 'side', 'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/orig', 'wizardv3mr': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/wizardv3mr', 'wizardv3': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/wizardv3', 'cattouch': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/cattouch', 'small': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/small', '320x240': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/320x240', '1200x900': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/1200x900', '1200x900n': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/1200x900n', 'thumb_m': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/thumb_m', 'full': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/full', '832x624': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/832x624', '456x342': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/456x342', '120x90': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/120x90', '92x69': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/92x69', 'islandoff': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/islandoff', 'thumb_s': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/thumb_s', 'thumb_s_2x': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/thumb_s_2x', 'auto_main': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/auto_main', 'image': '//avatars.mds.yandex.net/get-verba/1030388/2a0000017840aee4306c54cca908e3246a7d/image'}}, {'name': 'front', 'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/orig', 'wizardv3mr': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/wizardv3mr', 'wizardv3': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/wizardv3', 'cattouch': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/cattouch', 'small': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/small', '320x240': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/320x240', '1200x900': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/1200x900', '1200x900n': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/1200x900n', 'thumb_m': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/thumb_m', 'full': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/full', '832x624': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/832x624', '456x342': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/456x342', '120x90': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/120x90', '92x69': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/92x69', 'islandoff': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/islandoff', 'thumb_s': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/thumb_s', 'thumb_s_2x': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/thumb_s_2x', 'auto_main': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/auto_main', 'image': '//avatars.mds.yandex.net/get-verba/3587101/2a0000017840aef2d929e2e2c6852e031f17/image'}}, {'name': 'back', 'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/orig', 'wizardv3mr': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/wizardv3mr', 'wizardv3': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/wizardv3', 'cattouch': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/cattouch', 'small': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/small', '320x240': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/320x240', '1200x900': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/1200x900', '1200x900n': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/1200x900n', 'thumb_m': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/thumb_m', 'full': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/full', '832x624': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/832x624', '456x342': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/456x342', '120x90': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/120x90', '92x69': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/92x69', 'islandoff': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/islandoff', 'thumb_s': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/thumb_s', 'thumb_s_2x': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/thumb_s_2x', 'auto_main': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/auto_main', 'image': '//avatars.mds.yandex.net/get-verba/216201/2a0000017840af005837ea5ccd5b6c4ae2fd/image'}}], 'main_color': True}]}
display(list(d))
d['id']
d['name']
d['available_options']
d['vendor_colors']

['id', 'name', 'available_options', 'vendor_colors']

[{'body_color_id': 22805390,
  'mark_color_id': 22191469,
  'name_ru': 'Designo Красная патагония',
  'hex_codes': ['ED173D'],
  'color_type': 'METALLIC',
  'stock_color': {'hex_code': 'EE1D19', 'name_ru': 'красный'},
  'photos': [{'name': '34-front',
    'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/orig',
     'wizardv3mr': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/wizardv3mr',
     'wizardv3': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/wizardv3',
     'cattouch': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/cattouch',
     'small': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/small',
     '320x240': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59/320x240',
     '1200x900': '//avatars.mds.yandex.net/get-verba/1540742/2a0000017840aec23ff26a655ac32ecf4e59

In [302]:
# смотрим пример словаря для description
df['description'][0]

'КАСКАД-АВТО - официальный дилер Mercedes-Benz\nпредлагает Вашему вниманию:\nMercedes-Benz E 200 4MATIC Sport Купе, 2020 года выпуска \n\nВы можете воспользоваться финансовыми программами от Мерседес-Бенц Банка: \n- приобрести авто в кредит по ставке от 6,5% годовых \n- приобрести авто в лизинг \n- оформить КАСКО от 3,5% от стоимости авто\n4 ГОДА ГАРАНТИИ!\n\n\nКОМПЛЕКТАЦИЯ: \nОтделение для мелких предметов\nИнструкция по эксплуатации и сервисная книжка на русском языке\nCONVERSION WORLD MANUFACTURER TO MERCEDES-BENZ AG\nПодготовка под Mercedes-Benz Link\nСистема мониторинга "слепых" зон\nЭлектрическая регулировка сиденья переднего пассажира с функцией памяти\nВнутреннее и боковые зеркала заднего вида с автоматическим затемнением\nСистема экстренного торможения\nGPS антенна\nПакет запоминания параметров\nСкладывающиеся спинки задних сидений\nСистема аварийного вызова "Мерседес-Бенц"\nПодготовка под установку навигационного модуля\nКоммуникационный модуль LTE\nПодготовка для установки L

In [308]:
# смотрим пример словаря для equipment_dict
df['equipment_dict'][0]
# содержаение первой записи
e = {'cruise-control': True, 'asr': True, 'esp': True, 'usb': True, 'e-adjustment-wheel': True, 'multi-wheel': True, 'auto-park': True, 'audiosystem-tv': True, 'airbag-passenger': True, 'front-centre-armrest': True, 'navigation': True, 'apple-carplay': True, 'front-seat-support': True, 'decorative-interior-lighting': True, 'isofix-front': True, 'bas': True, 'rear-camera': True, 'lock': True, 'servo': True, 'electro-mirrors': True, 'drl': True, 'electro-window-back': True, 'mirrors-heat': True, 'volume-sensor': True, 'rear-seats-heat': True, 'leather': True, 'seat-memory': True, '19-inch-wheels': True, 'driver-seat-updown': True, 'led-lights': True, 'engine-proof': True, 'glonass': True, 'start-stop-function': True, 'airbag-driver': True, 'isofix': True, 'aux': True, 'drive-mode-sys': True, 'electro-window-front': True, 'automatic-lighting-control': True, 'airbag-curtain': True, 'collision-prevention-assist': True, 'computer': True, 'high-beam-assist': True, 'seat-transformation': True, 'android-auto': True, 'passenger-seat-electric': True, 'alarm': True, 'alloy-wheel-disks': True, 'paint-metallic': True, 'leather-gear-stick': True, 'blind-spot': True, 'start-button': True, 'airbag-side': True, 'tyre-pressure': True, 'voice-recognition': True, 'audiosystem-cd': True, 'electronic-gage-panel': True, 'electro-trunk': True, 'abs': True, 'front-seats-heat': True, 'bluetooth': True, 'wheel-leather': True, 'immo': True, 'climate-control-2': True, 'auto-mirrors': True, '12v-socket': True, 'third-rear-headrest': True, 'black-roof': True}
display(list(e))


['cruise-control',
 'asr',
 'esp',
 'usb',
 'e-adjustment-wheel',
 'multi-wheel',
 'auto-park',
 'audiosystem-tv',
 'airbag-passenger',
 'front-centre-armrest',
 'navigation',
 'apple-carplay',
 'front-seat-support',
 'decorative-interior-lighting',
 'isofix-front',
 'bas',
 'rear-camera',
 'lock',
 'servo',
 'electro-mirrors',
 'drl',
 'electro-window-back',
 'mirrors-heat',
 'volume-sensor',
 'rear-seats-heat',
 'leather',
 'seat-memory',
 '19-inch-wheels',
 'driver-seat-updown',
 'led-lights',
 'engine-proof',
 'glonass',
 'start-stop-function',
 'airbag-driver',
 'isofix',
 'aux',
 'drive-mode-sys',
 'electro-window-front',
 'automatic-lighting-control',
 'airbag-curtain',
 'collision-prevention-assist',
 'computer',
 'high-beam-assist',
 'seat-transformation',
 'android-auto',
 'passenger-seat-electric',
 'alarm',
 'alloy-wheel-disks',
 'paint-metallic',
 'leather-gear-stick',
 'blind-spot',
 'start-button',
 'airbag-side',
 'tyre-pressure',
 'voice-recognition',
 'audiosystem-cd'

In [309]:
df['image']

0        https://autoru.naydex.net/te1Ec7847/f0ef6cnRuw...
1        https://avatars.mds.yandex.net/get-autoru-vos/...
2        https://autoru.naydex.net/te1Ec7847/f0ef6cnRuw...
3        https://autoru.naydex.net/te1Ec7847/f0ef6cnRuw...
4        https://autoru.naydex.net/te1Ec7847/f0ef6cnRuw...
                               ...                        
28668    https://autoru.naydex.net/8NspfV975/35ebceAKti...
28669    https://autoru.naydex.net/8NspfV975/35ebceAKti...
28670    https://avatars.mds.yandex.net/get-autoru-vos/...
28671    https://avatars.mds.yandex.net/get-autoru-vos/...
28672    https://autoru.naydex.net/8NspfV975/35ebceAKti...
Name: image, Length: 28673, dtype: object

In [310]:
df['priceCurrency'].value_counts()

RUB    28673
Name: priceCurrency, dtype: int64

In [311]:
df['sell_id'].value_counts()

1105842909-b2f0f68d    5
1106001416-b885f855    5
1106089334-8b06b9c0    4
1106108638-bb22eee3    4
1105988655-93361514    4
                      ..
1106092118-0b3e547a    1
1105904504-b8f978cb    1
1105747952-2ff48a5a    1
1106109182-95a69719    1
1105431058-302416de    1
Name: sell_id, Length: 25702, dtype: int64

In [317]:
# смотрим пример словаря для equipment_dict
df['super_gen'][0]

# содержаение первой записи
s = {'id': '22284156', 'name': 'V (W213, S213, C238) Рестайлинг', 'ru_name': '5 (W213, S213, C238) Рестайлинг', 'year_from': 2020, 'price_segment': 'PREMIUM'}
display(list(s))

['id', 'name', 'ru_name', 'year_from', 'price_segment']

In [319]:
df['super_gen'][5:10]

5    {'id': '7869981', 'year_from': 1985, 'year_to'...
6    {'id': '2305837', 'name': 'III', 'ru_name': '3...
7    {'id': '2307743', 'name': 'V (W221)', 'ru_name...
8    {'id': '7770222', 'name': 'IX Рестайлинг', 'ye...
9    {'id': '22913367', 'name': 'II', 'ru_name': '2...
Name: super_gen, dtype: object

In [325]:
# этот столбец состоит уже из 3х имеющихся - хз, нужен ли.....
df['vehicleConfiguration'].head()

0                COUPE AUTOMATIC 2.0
1      ALLROAD_5_DOORS AUTOMATIC 2.5
2               SEDAN MECHANICAL 1.6
3      ALLROAD_5_DOORS AUTOMATIC 2.0
4    HATCHBACK_5_DOORS AUTOMATIC 1.0
Name: vehicleConfiguration, dtype: object

In [321]:
df['car_url'].head()

0    https://auto.ru/cars/new/group/mercedes/e_klas...
1    https://auto.ru/cars/used/sale/toyota/rav_4/11...
2    https://auto.ru/cars/used/sale/renault/megane/...
3    https://auto.ru/cars/new/group/hyundai/creta/2...
4    https://auto.ru/cars/used/sale/toyota/vitz/110...
Name: car_url, dtype: object

In [25]:
df['price'].head()
df['price'].describe()

count    3.662000e+03
mean     1.651061e+06
std      1.264821e+06
min      2.500000e+04
25%      8.257500e+05
50%      1.250400e+06
75%      2.130225e+06
max      1.111720e+07
Name: price, dtype: float64

In [324]:
# этот столбец состоит уже из 2х имеющихся
df['name'].head()

0    2.0 AT
1    2.5 AT
2    1.6 MT
3    2.0 AT
4    1.0 AT
Name: name, dtype: object

In [ ]:
корреляция
есть ли переобучение
простую регрессию
катбуст
стекинг
